In [4]:
from functions import *
from ipywidgets import HBox, HTML, Dropdown, Label, Layout,Image
import tkinter as tk
from IPython.display import display, Markdown, clear_output, IFrame,Javascript
from pyvis.network import Network
pathways_name=pd.read_csv("data/pathways.tsv", sep='\t')["pathway_name"]
last_selection=pathways_name
display(HTML("<style>.container { width:100% !important; }</style>"))


HTML(value='<style>.container { width:100% !important; }</style>')

In [5]:
display(HTML("<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>"))
text=Label(value="")
text.add_class("left-spacing-class")
menu = Dropdown(
       options=list(pathways_name.values),
       value=list(pathways_name.values)[25],
       description='Pathway:')
sidebyside=HBox([menu, HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells([2])"><input type="submit" id="toggleButton" value="Run"></form>'''),text])

display(sidebyside)

HTML(value='<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>')

In [65]:
skip_calcs=False
selection=menu.value
pathway_edges=read_pathway(selection)
if (len(pathway_edges)==0):
    selection=last_selection
    skip_calcs=True
    st.error("Edges not found, try another pathway!")
else:
    skip_calcs=False
    last_selection=selection
    pathway_edges=read_pathway(selection)
    adj_matrix,nodes_renamed,inv_nodes_renamed=build_adj(pathway_edges)
    G = nx.from_numpy_matrix(adj_matrix)
    triad_cliques=get_triad(G)
    weighted_edges=calculate_weighted_edges(triad_cliques, adj_matrix,inv_nodes_renamed)
    to_remove=[]
    signify_values={}
    essential_edges=[]
    for x in weighted_edges.items():
        zeros=0
        ones=0
        minus=0
        for z in x[1]:
            if (z[1]==0):
                zeros+=1
            elif (z[1]==1):
                ones+=1
            else:
                minus+=1
        if (ones==0):
            if (minus==0):
                to_remove.append(x[0])
            else:
                m=(zeros+minus)/2
                if ((minus+zeros)/(zeros*minus+1)*zeros/(minus+zeros)>((minus+zeros)/(m*m+1))*zeros/(minus+zeros)):
                    to_remove.append(x[0])
        else:
            essential_edges.append(x[0])
        if (ones==0):
            signify_values[x[0]]=(minus+zeros)/(zeros*minus+1)*zeros/(minus+zeros)
        else:
            signify_values[x[0]]=0

    relabel={}
    for e,node in enumerate( G.nodes()):
        relabel[e]=str(inv_nodes_renamed[node])
net=Network(height="825px",notebook=True,directed=True,width="1800px", bgcolor='#222222', font_color='white')
for i,node in relabel.items():
    net.add_node(str(node))

for edge in pathway_edges.values:
        if(edge[2]==-1):
            net.add_edge(str(edge[0]), str(edge[1]), color="yellow")
        else:
            net.add_edge(str(edge[0]), str(edge[1]))
for triad in triad_cliques:
    for i,x in enumerate(triad):
        for j,y in enumerate(triad):
            value=""
            isessential=""
            min_node=min(triad[i],triad[j])
            max_node=max(triad[i],triad[j])
            weight=pathway_edges[(pathway_edges[0]==inv_nodes_renamed[min_node]) & (pathway_edges[1]==inv_nodes_renamed[max_node])]
            if (weight.empty):
                continue
            if ((str(inv_nodes_renamed[min_node])+","+str(inv_nodes_renamed[max_node])) in to_remove):           
                color="red"
                size=10
                value+=", significativity:  "+str(signify_values[str(inv_nodes_renamed[min_node])+","+str(inv_nodes_renamed[max_node])])
            else:
                color="green"
                size=3
                value+=", significativity:  "+str(signify_values[str(inv_nodes_renamed[min_node])+","+str(inv_nodes_renamed[max_node])])
            if ((str(inv_nodes_renamed[min_node])+","+str(inv_nodes_renamed[max_node])) in essential_edges):   
                isessential="Essential "
            weight=int(weight[2].values)
            if (weight==1):
                net.add_edge(str(inv_nodes_renamed[min_node]), str(inv_nodes_renamed[max_node]), color=color, width=size,title=isessential+"Expression edge"+value)
            else:
                net.add_edge(str(inv_nodes_renamed[min_node]), str(inv_nodes_renamed[max_node]), color=color, width=size,title=isessential+"Suppression edge"+value)
net.hrepulsion(node_distance=120, central_gravity=0.0, spring_length=100, spring_strength=0, damping=0.09)
net.show("data/graph.html")

In [8]:
signify_values

{'84126,57697': 1.0,
 '55215,57697': 1.0,
 '84126,55215': 0,
 '84126,2189': 1.0,
 '55215,2189': 0,
 '84126,2175': 1.0,
 '55215,2175': 2.0,
 '545,57697': 0,
 '545,55215': 0,
 '545,2189': 1.0,
 '545,2175': 1.0}

In [56]:
signify_values={}
for x in weighted_edges.items():
        zeros=0
        ones=0
        minus=0
        for z in x[1]:
            if (z[1]==0):
                zeros+=1
            elif (z[1]==1):
                ones+=1
            else:
                minus+=1
        if (ones==0):
            if (minus==0):
                to_remove.append(x[0])
            else:
                m=(zeros+minus)/2
                if ((minus+zeros)/(zeros*minus+1)*zeros/(minus+zeros)>((minus+zeros)/(m*m+1))*zeros/(minus+zeros)):
                    to_remove.append(x[0])
        else:
            essential_edges.append(x[0])
        if (ones==0):
            signify_values[x[0]]=(minus+zeros)*zeros/(zeros*minus+1)
        else:
            signify_values[x[0]]=0

In [59]:
weighted_edges

{'9564,1399': [(-0.0822772340460296, -1),
  (0.8309499100997029, 1),
  (0.311201120382225, -1),
  (0.0008240032064636366, 0),
  (0.5609662726712342, 1),
  (0.32362390097957927, -1),
  (-0.13280159063449845, -1)],
 '2185,1399': [(0.5485113749219565, -1),
  (0.7615167957411138, -1),
  (-0.10351418538932301, -1),
  (-0.33894683035124157, -1)],
 '9564,2185': [(0.5485113749219565, -1),
  (0.29571758867491255, -1),
  (0.5735792614209305, -1),
  (0.1747808164909129, -1),
  (0.494573051231267, -1),
  (0.7584949071286211, -1),
  (0.8652160668963143, -1),
  (0.23578678712209472, -1)],
 '9564,1398': [(-0.15993240874094317, -1),
  (0.7388926987492321, -1),
  (0.0793054304163893, -1),
  (0.3481647074804001, -1),
  (0.2736372228330657, -1),
  (0.7830375224623766, -1),
  (0.06578030887943524, -1)],
 '2185,1398': [(0.29571758867491255, -1),
  (0.6451933217666298, -1),
  (-0.11277834114323117, -1),
  (-0.015171424531481771, 0)],
 '9564,5747': [(-0.10927189397300595, -1),
  (0.007039154011118935, 0),
  

In [66]:
signify_values['1398,3689']

0.3333333333333333